In [105]:
import pandas as pd
import numpy as np 
import igraph as ig

path_list = []
with open("pathway_gene_sets/c2.cp.kegg.v2023.1.Hs.symbols.gmt") as gmt:
    for i in gmt:
        path_list.append([i.split()[0]] + i.split()[2:])


In [111]:
def get_sample_pair(sampleGroup):
    all_sample = []
    with open(f"../leave_one_out/{sampleGroup}.weight.txt") as f_w:
        for i in f_w:
            i= i.strip().split()
            all_sample.append(i[0])
    return all_sample

cate = "primaryEver"
sampleGroup = get_sample_pair(cate)

In [112]:
def read_g(sample):
    dftemp = pd.read_csv(f"../leave_one_out/all_sample_k_10/sample_specific_{sample}.txt",sep="\t",header=None)
    dftemp.columns = [0,1,"weight"]
    dftemp["weight"] = abs(dftemp["weight"])
    #dftemp.loc[dftemp["weight"]>2, "weight"] = 2
    #dftemp["weight"] = 2.01 - dftemp["weight"]
    ##### important to divide the pattern of postive and negative #######
    #dftemp = dftemp[dftemp["weight"]>0] #positive # negative
    #dftemp["weight"] =abs(dftemp["weight"]) 
    all_genes = set(dftemp[0].tolist() + dftemp[1].tolist())
    sGCN = ig.Graph.TupleList(dftemp.itertuples(index=False),edge_attrs="weight")
    return sGCN, all_genes
    

In [113]:
import math
def cal_entropy(alist):
    h = 0
    if sum(alist) != 0:
        temp = sum(alist)
    for i in alist:
        if i == 0:
            h += 0
        else:
            h += (i/temp) * math.log2(i/temp)
    return -h

In [114]:
def get_str(alist):
    return "_".join([str(i) for i in alist])

#our_dir = f"c2kegg_pathway_degree_entropy_{cate}"

ep_pathway = open(f"c2kegg_pathway_edge_entropy_{cate}.txt","w")
ep_pathway.write("sample"+"\t"+ "\t".join([ j[0] for j in path_list])+"\n")

for num,i in enumerate(sampleGroup):
    g = read_g(i)
    allg_genes = g[1]
    
    all_entropy = []
    for j in path_list:
        left_genes = set(j[1:]).intersection(allg_genes)
    ##extract subgraph based pathway genes
        subg = g[0].subgraph(left_genes)
        total, genes_nx = len(j[1:]), len(left_genes)
        #print([t for t in subg.es])
        if genes_nx > 1:
            #edgebe = subg.edge_betweenness(directed=False, cutoff=None, weights="weight")
            #print([es["weight"] for es in subg.es])
            pathentropy = cal_entropy([es["weight"] for es in subg.es])
            
            #print(eig, ecc, edgebe,clos, coren)
      #      f_pathway.write(j[0]+"\t"+str(total)+"\t"+str(genes_nx)+"\t"+ get_str(eig)+"\t"
      #                      +get_str(ecc)+"\t"+get_str(edgebe)+"\t"+get_str(clos)+"\t"+get_str(coren)+"\n")
        else:
            pathentropy = 0
     #       
            pass
        all_entropy.append(pathentropy)
    ep_pathway.write(i+"\t"+ "\t".join( [ str(indivi) for indivi in all_entropy] )+"\n")
    #f_pathway.close()
    print(f"completed rate {num*100/len(sampleGroup)}%",end="\n",flush=True)

completed rate 0.0%
completed rate 1.0869565217391304%
completed rate 2.1739130434782608%
completed rate 3.260869565217391%
completed rate 4.3478260869565215%
completed rate 5.434782608695652%
completed rate 6.521739130434782%
completed rate 7.608695652173913%
completed rate 8.695652173913043%
completed rate 9.782608695652174%
completed rate 10.869565217391305%
completed rate 11.956521739130435%
completed rate 13.043478260869565%
completed rate 14.130434782608695%
completed rate 15.217391304347826%
completed rate 16.304347826086957%
completed rate 17.391304347826086%
completed rate 18.47826086956522%
completed rate 19.565217391304348%
completed rate 20.652173913043477%
completed rate 21.73913043478261%
completed rate 22.82608695652174%
completed rate 23.91304347826087%
completed rate 25.0%
completed rate 26.08695652173913%
completed rate 27.17391304347826%
completed rate 28.26086956521739%
completed rate 29.347826086956523%
completed rate 30.434782608695652%
completed rate 31.521739130

In [115]:
ep_pathway.close()